#**Problem Statement**  
### **Business problem overview**  
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.   

For many incumbent operators, retaining high profitable customers is the number one business goal.   

To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.   

In this project, you will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.   

#### **Understanding and defining churn**
There are two main models of payment in the telecom industry - postpaid (customers pay a monthly/annual bill after using the services) and prepaid (customers pay/recharge with a certain amount in advance and then use the services).  

In the postpaid model, when customers want to switch to another operator, they usually inform the existing operator to terminate the services, and you directly know that this is an instance of churn.   

However, in the prepaid model, customers who want to switch to another network can simply stop using the services without any notice, and it is hard to know whether someone has actually churned or is simply not using the services temporarily (e.g. someone may be on a trip abroad for a month or two and then intend to resume using the services again).    

Thus, churn prediction is usually more critical (and non-trivial) for prepaid customers, and the term ‘churn’ should be defined carefully.  Also, prepaid is the most common model in India and Southeast Asia, while postpaid is more common in Europe in North America.  

This project is based on the Indian and Southeast Asian market.   

#### **Definitions of churn**  
There are various ways to define churn, such as:
**Revenue-based churn:** Customers who have not utilised any revenue-generating facilities such as mobile internet, outgoing calls, SMS etc. over a given period of time. One could also use aggregate metrics such as ‘customers who have generated less than INR 4 per month in total/average/median revenue’.   

The main shortcoming of this definition is that there are customers who only receive calls/SMSes from their wage-earning counterparts, i.e. they don’t generate revenue but use the services. For example, many users in rural areas only receive calls from their wage-earning siblings in urban areas.   

**Usage-based churn:** Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc. over a period of time.  

A potential shortcoming of this definition is that when the customer has stopped using the services for a while, it may be too late to take any corrective actions to retain them. For e.g., if you define churn based on a ‘two-months zero usage’ period, predicting churn could be useless since by that time the customer would have already switched to another operator.   

In this project, you will use the usage-based definition to define churn.   

#### **High-value churn**
In the Indian and the Southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.   

In this project, you will define high-value customers based on a certain metric (mentioned later below) and predict churn only on high-value customers.   

##### **Understanding the business objective and the data**  
The dataset contains customer-level information for a span of four consecutive months - June, July, August and September. The months are encoded as 6, 7, 8 and 9, respectively.  
The business objective is to predict the churn in the last (i.e. the ninth) month using the data (features) from the first three months. To do this task well, understanding the typical customer behaviour during churn will be helpful.   

##### **Understanding customer behaviour during churn**  
Customers usually do not decide to switch to another competitor instantly, but rather over a period of time (this is especially applicable to high-value customers). In churn prediction, we assume that there are three phases of customer lifecycle :  
The ‘good’ phase: In this phase, the customer is happy with the service and behaves as usual.  

The ‘action’ phase: The customer experience starts to sore in this phase, for e.g. he/she gets a compelling offer from a  competitor, faces unjust charges, becomes unhappy with service quality etc. In this phase, the customer usually shows different behaviour than the ‘good’ months. Also, it is crucial to identify high-churn-risk customers in this phase, since some corrective actions can be taken at this point (such as matching the competitor’s offer/improving the service quality etc.)  

The ‘churn’ phase: In this phase, the customer is said to have churned. You define churn based on this phase. Also, it is important to note that at the time of prediction (i.e. the action months), this data is not available to you for prediction. Thus, after tagging churn as 1/0 based on this phase, you discard all data corresponding to this phase.   

In this case, since you are working over a four-month window, the first two months are the ‘good’ phase, the third month is the ‘action’ phase, while the fourth month is the ‘churn’ phase.


# **Reading and Understanding the Data**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

In [2]:
# Reading the data
churn = pd.read_csv("telecom_churn_data.csv")
churn.head(10)

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_ic_mou_9,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,roam_og_mou_9,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2t_mou_9,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2m_mou_9,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2f_mou_9,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_t2c_mou_9,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,loc_og_mou_9,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2t_mou_9,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2m_mou_9,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2f_mou_9,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_t2c_mou_9,std_og_mou_6,std_og_mou_7,std_og_mou_8,std_og_mou_9,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,isd_og_mou_9,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,spl_og_mou_9,og_others_6,og_others_7,og_others_8,og_others_9,total_og_mou_6,total_og_mou_7,total_og_mou_8,total_og_mou_9,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2t_mou_9,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2m_mou_9,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_t2f_mou_9,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,loc_ic_mou_9,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2t_mou_9,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2m_mou_9,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2f_mou_9,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_t2o_mou_9,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,std_ic_mou_9,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,total_ic_mou_9,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,spl_ic_mou_9,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,isd_ic_mou_9,ic_others_6,ic_others_7,ic_others_8,ic_others_9,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_num_9,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,total_rech_amt_9,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,last_day_rch_amt_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9,total_rech_data_6,total_rech_data_7,total_rech_data_8,total_rech_data_9,max_rech_data_6,max_rech_data_7,max_rech_data_8,max_rech_data_9,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_2g_9,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,count_rech_3g_9,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,av_rech_amt_data_9,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_2g_mb_9,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,vol_3g_mb_9,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_3g_9,arpu_2g_6,arpu_2g_7,arpu_2g_8,arpu_2g_9,night_pck_user_6,night_pck_user_7,night_pck_user_8,night_pck_user_9,monthly_2g_6,monthly_2g_7,monthly_2g_8,monthly_2g_9,sachet_2g_6,sachet_2g_7,sachet_2g_8,sachet_2g_9,monthly_3g_6,monthly_3g_7,monthly_3g_8,monthly_3g_9,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,214.816,213.803,21.100,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.16,NaN,NaN,NaN,4.13,NaN,NaN,NaN,1.15,NaN,NaN,NaN,5.44,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00,0.00,5.44,0.00,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,

In [3]:
# Lets check the dimensions of the dataset
churn.shape

(71547, 226)

In [4]:
#Verifying column datatypes
churn.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 226 columns):
 #    Column                    Non-Null Count  Dtype  
---   ------                    --------------  -----  
 0    mobile_number             71547 non-null  int64  
 1    circle_id                 71547 non-null  int64  
 2    loc_og_t2o_mou            70811 non-null  float64
 3    std_og_t2o_mou            70811 non-null  float64
 4    loc_ic_t2o_mou            70811 non-null  float64
 5    last_date_of_month_6      71547 non-null  object 
 6    last_date_of_month_7      71102 non-null  object 
 7    last_date_of_month_8      70752 non-null  object 
 8    last_date_of_month_9      70357 non-null  object 
 9    arpu_6                    71547 non-null  float64
 10   arpu_7                    71547 non-null  float64
 11   arpu_8                    71547 non-null  float64
 12   arpu_9                    71547 non-null  float64
 13   onnet_mou_6               68744 non-null  fl

In [5]:
last_date_cols = ['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9']

In [6]:
last_recharge_cols = ['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9']

In [7]:
date_of_last_recharge = ['date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8','date_of_last_rech_data_9']

In [8]:
churn[date_of_last_recharge].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   date_of_last_rech_data_6  17951 non-null  object
 1   date_of_last_rech_data_7  18210 non-null  object
 2   date_of_last_rech_data_8  18808 non-null  object
 3   date_of_last_rech_data_9  18465 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


In [9]:
churn[last_date_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   last_date_of_month_6  71547 non-null  object
 1   last_date_of_month_7  71102 non-null  object
 2   last_date_of_month_8  70752 non-null  object
 3   last_date_of_month_9  70357 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


In [10]:
churn[last_recharge_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date_of_last_rech_6  70395 non-null  object
 1   date_of_last_rech_7  70273 non-null  object
 2   date_of_last_rech_8  68971 non-null  object
 3   date_of_last_rech_9  68171 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


In [11]:
churn[last_recharge_cols]

,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9
0,6/21/2014,7/16/2014,8/8/2014,9/28/2014
1,6/29/2014,7/31/2014,8/28/2014,9/30/2014
2,6/17/2014,7/24/2014,8/14/2014,9/29/2014
3,6/28/2014,7/31/2014,8/31/2014,9/30/2014
4,6/26/2014,7/28/2014,8/9/2014,9/28/2014
...,...,...,...,...
71542,6/28/2014,7/20/2014,8/26/2014,9/5/2014
71543,6/27/2014,7/30/2014,8/31/2014,9/29/2014
71544,6/30/2014,7/31/2014,8/10/2014,9/20/2014
71545,6/27/2014,7/27/2014,8/25/2014,9/23/2014


In [12]:
churn[last_date_cols]

,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9
0,6/30/2014,7/31/2014,8/31/2014,9/30/2014
1,6/30/2014,7/31/2014,8/31/2014,9/30/2014
2,6/30/2014,7/31/2014,8/31/2014,9/30/2014
3,6/30/2014,7/31/2014,8/31/2014,9/30/2014
4,6/30/2014,7/31/2014,8/31/2014,9/30/2014
...,...,...,...,...
71542,6/30/2014,7/31/2014,8/31/2014,9/30/2014
71543,6/30/2014,7/31/2014,8/31/2014,9/30/2014
71544,6/30/2014,7/31/2014,8/31/2014,9/30/2014
71545,6/30/2014,7/31/2014,8/31/2014,9/30/2014


In [13]:
pd.to_datetime(churn['last_date_of_month_6'],format='%m/%d/%Y')

0       2014-06-30
1       2014-06-30
2       2014-06-30
3       2014-06-30
4       2014-06-30
           ...    
71542   2014-06-30
71543   2014-06-30
71544   2014-06-30
71545   2014-06-30
71546   2014-06-30
Name: last_date_of_month_6, Length: 71547, dtype: datetime64[ns]

In [14]:
for i in last_date_cols:
  churn[i] = pd.to_datetime(churn[i],format='%m/%d/%Y')

In [15]:
for i in last_recharge_cols:
  churn[i] = pd.to_datetime(churn[i],format='%m/%d/%Y')

In [16]:
for i in date_of_last_recharge:
  churn[i] = pd.to_datetime(churn[i],format='%m/%d/%Y')

In [17]:
churn[last_date_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   last_date_of_month_6  71547 non-null  datetime64[ns]
 1   last_date_of_month_7  71102 non-null  datetime64[ns]
 2   last_date_of_month_8  70752 non-null  datetime64[ns]
 3   last_date_of_month_9  70357 non-null  datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 2.2 MB


In [18]:
churn[last_recharge_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date_of_last_rech_6  70395 non-null  datetime64[ns]
 1   date_of_last_rech_7  70273 non-null  datetime64[ns]
 2   date_of_last_rech_8  68971 non-null  datetime64[ns]
 3   date_of_last_rech_9  68171 non-null  datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 2.2 MB


In [19]:
churn[last_date_cols+last_recharge_cols+date_of_last_recharge].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   last_date_of_month_6      71547 non-null  datetime64[ns]
 1   last_date_of_month_7      71102 non-null  datetime64[ns]
 2   last_date_of_month_8      70752 non-null  datetime64[ns]
 3   last_date_of_month_9      70357 non-null  datetime64[ns]
 4   date_of_last_rech_6       70395 non-null  datetime64[ns]
 5   date_of_last_rech_7       70273 non-null  datetime64[ns]
 6   date_of_last_rech_8       68971 non-null  datetime64[ns]
 7   date_of_last_rech_9       68171 non-null  datetime64[ns]
 8   date_of_last_rech_data_6  17951 non-null  datetime64[ns]
 9   date_of_last_rech_data_7  18210 non-null  datetime64[ns]
 10  date_of_last_rech_data_8  18808 non-null  datetime64[ns]
 11  date_of_last_rech_data_9  18465 non-null  datetime64[ns]
dtypes: datetime64[ns](

In [20]:
churn[last_date_cols+last_recharge_cols+date_of_last_recharge].head()

,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9
0,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-06-21,2014-07-16,2014-08-08,2014-09-28,2014-06-21,2014-07-16,2014-08-08,NaT
1,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-06-29,2014-07-31,2014-08-28,2014-09-30,NaT,2014-07-25,2014-08-10,NaT
2,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-06-17,2014-07-24,2014-08-14,2014-09-29,NaT,NaT,NaT,2014-09-17
3,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-06-28,2014-07-31,2014-08-31,2014-09-30,NaT,NaT,NaT,NaT
4,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-06-26,2014-07-28,2014-08-09,2014-09-28,2014-06-04,NaT,NaT,NaT


In [21]:
for i in churn.columns:
  print("*****")
  print(i)
  print(churn[i].value_counts(dropna=False))
  print("*****")

Streaming output truncated to the last 5000 lines.
Name: loc_ic_t2t_mou_9, Length: 11326, dtype: int64
*****
*****
loc_ic_t2m_mou_6
0.00      4473
NaN       2803
0.30        35
0.56        29
0.96        29
          ... 
259.91       1
346.04       1
409.39       1
135.91       1
663.91       1
Name: loc_ic_t2m_mou_6, Length: 19254, dtype: int64
*****
*****
loc_ic_t2m_mou_7
0.00      4068
NaN       2766
0.93        32
0.21        28
2.36        27
          ... 
301.88       1
151.83       1
13.35        1
81.71        1
472.69       1
Name: loc_ic_t2m_mou_7, Length: 19212, dtype: int64
*****
*****
loc_ic_t2m_mou_8
0.00      3897
NaN       3785
0.33        29
0.46        28
0.41        27
          ... 
226.59       1
470.74       1
47.25        1
272.54       1
619.16       1
Name: loc_ic_t2m_mou_8, Length: 19127, dtype: int64
*****
*****
loc_ic_t2m_mou_9
NaN       5501
0.00      3769
0.38        28
0.46        27
0.56        27
          ... 
335.53       1
310.18       1
179.58    

In [22]:
count_fb_cols = ['fb_user_6','fb_user_7','fb_user_8','fb_user_9','count_rech_2g_6','count_rech_2g_7','count_rech_2g_8','count_rech_2g_9','count_rech_3g_6','count_rech_3g_7','count_rech_3g_8','count_rech_3g_9']

In [23]:
# for i in count_fb_cols:
#   churn[i] = churn[i].astype('int64',errors='ignore')

In [24]:
# Checking dataset
churn.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71547 entries, 0 to 71546
Data columns (total 226 columns):
 #    Column                    Non-Null Count  Dtype         
---   ------                    --------------  -----         
 0    mobile_number             71547 non-null  int64         
 1    circle_id                 71547 non-null  int64         
 2    loc_og_t2o_mou            70811 non-null  float64       
 3    std_og_t2o_mou            70811 non-null  float64       
 4    loc_ic_t2o_mou            70811 non-null  float64       
 5    last_date_of_month_6      71547 non-null  datetime64[ns]
 6    last_date_of_month_7      71102 non-null  datetime64[ns]
 7    last_date_of_month_8      70752 non-null  datetime64[ns]
 8    last_date_of_month_9      70357 non-null  datetime64[ns]
 9    arpu_6                    71547 non-null  float64       
 10   arpu_7                    71547 non-null  float64       
 11   arpu_8                    71547 non-null  float64       
 12   ar

### Missing Value Imputation 

In [25]:
# Checking missing values in the dataset
(100*churn.isnull().sum()/len(churn)).sort_values(ascending=False)

arpu_3g_6                   74.910199
night_pck_user_6            74.910199
total_rech_data_6           74.910199
arpu_2g_6                   74.910199
max_rech_data_6             74.910199
fb_user_6                   74.910199
av_rech_amt_data_6          74.910199
date_of_last_rech_data_6    74.910199
count_rech_2g_6             74.910199
count_rech_3g_6             74.910199
date_of_last_rech_data_7    74.548199
total_rech_data_7           74.548199
fb_user_7                   74.548199
max_rech_data_7             74.548199
night_pck_user_7            74.548199
count_rech_2g_7             74.548199
av_rech_amt_data_7          74.548199
arpu_2g_7                   74.548199
count_rech_3g_7             74.548199
arpu_3g_7                   74.548199
total_rech_data_9           74.191790
count_rech_3g_9             74.191790
fb_user_9                   74.191790
max_rech_data_9             74.191790
arpu_3g_9                   74.191790
date_of_last_rech_data_9    74.191790
night_pck_us

In [26]:
# Checking Missing Values and treating them

In [27]:
churn[['total_rech_data_6','av_rech_amt_data_6','max_rech_data_6','count_rech_2g_6','count_rech_3g_6','night_pck_user_6','arpu_3g_6','arpu_2g_6','fb_user_6','date_of_last_rech_data_6']].isnull().sum()

total_rech_data_6           53596
av_rech_amt_data_6          53596
max_rech_data_6             53596
count_rech_2g_6             53596
count_rech_3g_6             53596
night_pck_user_6            53596
arpu_3g_6                   53596
arpu_2g_6                   53596
fb_user_6                   53596
date_of_last_rech_data_6    53596
dtype: int64

In [28]:
churn[['total_rech_data_7','av_rech_amt_data_7','max_rech_data_7','count_rech_2g_7','count_rech_3g_7','night_pck_user_7','arpu_3g_7','arpu_2g_7','fb_user_7','date_of_last_rech_data_7']].isnull().sum()

total_rech_data_7           53337
av_rech_amt_data_7          53337
max_rech_data_7             53337
count_rech_2g_7             53337
count_rech_3g_7             53337
night_pck_user_7            53337
arpu_3g_7                   53337
arpu_2g_7                   53337
fb_user_7                   53337
date_of_last_rech_data_7    53337
dtype: int64

In [29]:
churn[['total_rech_data_8','av_rech_amt_data_8','max_rech_data_8','count_rech_2g_8','count_rech_3g_8','night_pck_user_8','arpu_3g_8','arpu_2g_8','fb_user_8','date_of_last_rech_data_8']].isnull().sum()

total_rech_data_8           52739
av_rech_amt_data_8          52739
max_rech_data_8             52739
count_rech_2g_8             52739
count_rech_3g_8             52739
night_pck_user_8            52739
arpu_3g_8                   52739
arpu_2g_8                   52739
fb_user_8                   52739
date_of_last_rech_data_8    52739
dtype: int64

In [30]:
churn[['total_rech_data_9','av_rech_amt_data_9','max_rech_data_9','count_rech_2g_9','count_rech_3g_9','night_pck_user_9','arpu_3g_9','arpu_2g_9','fb_user_9','date_of_last_rech_data_9']].isnull().sum()

total_rech_data_9           53082
av_rech_amt_data_9          53082
max_rech_data_9             53082
count_rech_2g_9             53082
count_rech_3g_9             53082
night_pck_user_9            53082
arpu_3g_9                   53082
arpu_2g_9                   53082
fb_user_9                   53082
date_of_last_rech_data_9    53082
dtype: int64

In [31]:
#From above we can see that id the last recharge date is not available for that month , then all the other recharge related data is null. So we will impute the recharge related field with 0

In [32]:
rechargeRelatedFields = ['total_rech_data_6','av_rech_amt_data_6','max_rech_data_6','count_rech_2g_6','count_rech_3g_6','night_pck_user_6','arpu_3g_6','arpu_2g_6','fb_user_6',
'total_rech_data_7','av_rech_amt_data_7','max_rech_data_7','count_rech_2g_7','count_rech_3g_7','night_pck_user_7','arpu_3g_7','arpu_2g_7','fb_user_7',
'total_rech_data_8','av_rech_amt_data_8','max_rech_data_8','count_rech_2g_8','count_rech_3g_8','night_pck_user_8','arpu_3g_8','arpu_2g_8','fb_user_8',
'total_rech_data_9','av_rech_amt_data_9','max_rech_data_9','count_rech_2g_9','count_rech_3g_9','night_pck_user_9','arpu_3g_9','arpu_2g_9','fb_user_9']

In [33]:
churn.update(churn[rechargeRelatedFields].fillna(0))

In [34]:
churn[rechargeRelatedFields].isnull().sum()

total_rech_data_6     0
av_rech_amt_data_6    0
max_rech_data_6       0
count_rech_2g_6       0
count_rech_3g_6       0
night_pck_user_6      0
arpu_3g_6             0
arpu_2g_6             0
fb_user_6             0
total_rech_data_7     0
av_rech_amt_data_7    0
max_rech_data_7       0
count_rech_2g_7       0
count_rech_3g_7       0
night_pck_user_7      0
arpu_3g_7             0
arpu_2g_7             0
fb_user_7             0
total_rech_data_8     0
av_rech_amt_data_8    0
max_rech_data_8       0
count_rech_2g_8       0
count_rech_3g_8       0
night_pck_user_8      0
arpu_3g_8             0
arpu_2g_8             0
fb_user_8             0
total_rech_data_9     0
av_rech_amt_data_9    0
max_rech_data_9       0
count_rech_2g_9       0
count_rech_3g_9       0
night_pck_user_9      0
arpu_3g_9             0
arpu_2g_9             0
fb_user_9             0
dtype: int64

In [ ]:
# Now will check for mou data

In [35]:
(100*churn.isnull().sum()/len(churn)).sort_values(ascending=False)

date_of_last_rech_data_6    74.910199
date_of_last_rech_data_7    74.548199
date_of_last_rech_data_9    74.191790
date_of_last_rech_data_8    73.712385
std_og_t2f_mou_9             7.688652
loc_ic_t2m_mou_9             7.688652
loc_og_t2f_mou_9             7.688652
loc_ic_mou_9                 7.688652
roam_ic_mou_9                7.688652
std_ic_t2o_mou_9             7.688652
std_og_t2t_mou_9             7.688652
std_og_t2m_mou_9             7.688652
roam_og_mou_9                7.688652
std_ic_t2f_mou_9             7.688652
std_og_t2c_mou_9             7.688652
spl_og_mou_9                 7.688652
loc_og_t2t_mou_9             7.688652
loc_og_t2m_mou_9             7.688652
offnet_mou_9                 7.688652
loc_ic_t2t_mou_9             7.688652
std_ic_mou_9                 7.688652
loc_ic_t2f_mou_9             7.688652
onnet_mou_9                  7.688652
std_ic_t2t_mou_9             7.688652
spl_ic_mou_9                 7.688652
std_og_mou_9                 7.688652
og_others_9 

In [36]:
# This method will replace the values with 0 for each month and each category
def handle_mou(df, category):
    for month in ['6','7','8','9']:
        mou_cols = [col for col in df.columns if ((col[-5:] == 'mou_' + month) & (category in col))]
        mou_cols.append('onnet_mou_' + month)
        mou_cols.append('offnet_mou_' + month)       
        total_col = 'total_' + category + '_mou_' + month
        df[mou_cols][df[total_col] == 0]
        df.update(df[df[total_col] == 0][mou_cols].fillna(0))

In [37]:
handle_mou(churn, 'og')
handle_mou(churn, 'ic')

In [38]:
(100*churn.isnull().sum()/len(churn)).sort_values(ascending=False)

date_of_last_rech_data_6    74.910199
date_of_last_rech_data_7    74.548199
date_of_last_rech_data_9    74.191790
date_of_last_rech_data_8    73.712385
ic_others_9                  7.688652
og_others_9                  7.688652
og_others_8                  5.290229
ic_others_8                  5.290229
date_of_last_rech_9          4.718577
og_others_6                  3.917704
ic_others_6                  3.917704
ic_others_7                  3.865990
og_others_7                  3.865990
date_of_last_rech_8          3.600430
date_of_last_rech_7          1.780648
last_date_of_month_9         1.663242
date_of_last_rech_6          1.610130
last_date_of_month_8         1.111158
loc_ic_t2o_mou               1.028694
loc_og_t2o_mou               1.028694
std_og_t2o_mou               1.028694
last_date_of_month_7         0.621969
vol_3g_mb_9                  0.001398
monthly_2g_9                 0.001398
monthly_2g_6                 0.001398
monthly_2g_7                 0.001398
monthly_2g_8

In [ ]:
# check stastical summary of the dataset
churn.describe().T

,count,mean,std,min,25%,50%,75%,max
mobile_number,99999.0,7.001207e+09,695669.386290,7.000000e+09,7.000606e+09,7.001205e+09,7.001812e+09,7.002411e+09
circle_id,99999.0,1.090000e+02,0.000000,1.090000e+02,1.090000e+02,1.090000e+02,1.090000e+02,1.090000e+02
loc_og_t2o_mou,98981.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
std_og_t2o_mou,98981.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
loc_ic_t2o_mou,98981.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
arpu_6,99999.0,2.829874e+02,328.439770,-2.258709e+03,9.341150e+01,1.977040e+02,3.710600e+02,2.773109e+04
arpu_7,99999.0,2.785366e+02,338.156291,-2.014045e+03,8.698050e+01,1.916400e+02,3.653445e+02,3.514583e+04
arpu_8,99999.0,2.791547e+02,344.474791,-9.458080e+02,8.412600e+01,1.920800e+02,3.693705e+02,3.354362e+04
arpu_9,99999.0,2.616451e+02,341.998630,-1.899505e+03,6.268500e+01,1.768490e+02,3.534665e+02,3.880562e+04
onnet_mou_6,96062.0,1.323959e+02,297.207406,0.000000e+00,7.380000e+00,3.431000e+01,1.187400e+02,7.376710e+03


In [ ]:
# Dropping all the columns having more than 40% missing values
drop_list = churn.columns[round(100*churn.isnull().sum()/len(churn),2).values > 50]
print(len(drop_list))
print(drop_list)

# Drop this columns
churn.drop(drop_list, axis=1,inplace=True)

40
Index(['date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8', 'date_of_last_rech_data_9', 'total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9', 'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9', 'count_rech_2g_6', 'count_rech_2g_7', 'count_rech_2g_8', 'count_rech_2g_9', 'count_rech_3g_6', 'count_rech_3g_7', 'count_rech_3g_8', 'count_rech_3g_9', 'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9', 'arpu_3g_6', 'arpu_3g_7', 'arpu_3g_8', 'arpu_3g_9', 'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8', 'arpu_2g_9', 'night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8', 'night_pck_user_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9'], dtype='object')


In [ ]:
# Lets check the shape of the dataset after deleting the columns
churn.shape


(99999, 186)